In [1]:
import pandas as pd
import os
import psycopg2
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [14]:
# Cria conexão com o banco de dados
def conexao():
    con = psycopg2.connect(
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        database=os.getenv("POSTGRES_DB"),
        host="localhost",
    )

    return con

# Cria função para selecionar os dados
def se_dados(query):
    vcon = conexao()
    c = vcon.cursor()
    c.execute(query)
    rows = c.fetchall()
    vcon.commit()
    vcon.close()
    return rows                    

In [15]:
# Cria query para gerar o relatório
query = f" SELECT DISTINCT papel, div_liquida, cotacao, pl, vpa, ev_ebitda, marg_liquida, lucro_liquido_12m, lucro_liquido_3m \
            FROM dados \
                WHERE data_dado_inserido >= '02-03-2022' \
                    AND div_liquida < '0' \
                    AND pl > 5 \
                    AND pl < 15 \
                        ORDER BY 2 "
result = se_dados(query)

# Cria DataFrame e salva o arquivo gerado em csv
df_rl = pd.DataFrame(result)
# Renomea o nome das colunas do Dataframe
df_rl.rename(columns={0:'Papel', 
    1:'Dívida Líquida', 
    2:'Cotação',
    3:'PL',
    4:'VPA',
    5:'EV/Ebitda',
    6:'Margem Líquida',
    7:'Lucro Líquido 12m',
    8:'Lucro Líquido 3m'}, inplace=True)
df_rl.to_csv('../Analise/relatorio.csv', sep=';', encoding='ISO-8859-1')
df_rl.head(7)

,Papel,Dívida Líquida,Cotação,PL,VPA,EV/Ebitda,Margem Líquida,Lucro Líquido 12m,Lucro Líquido 3m
0,PSSA3,"-R$ 8.877.880.000,00","R$ 20,93",8.76,14.48,0.27,7.3,"R$ 1.544.250.000,00","R$ 532.806.000,00"
1,CMIN3,"-R$ 7.392.710.000,00","R$ 6,48",5.17,2.55,2.27,33.0,"R$ 7.009.350.000,00","R$ 804.065.000,00"
2,TIMS3,"-R$ 5.951.170.000,00","R$ 13,45",11.01,10.37,2.97,16.4,"R$ 2.957.170.000,00","R$ 1.015.300.000,00"
3,ENAT3,"-R$ 1.506.960.000,00","R$ 18,28",6.14,13.04,3.29,60.7,"R$ 791.935.000,00","R$ 133.953.000,00"
4,WHRL3,"-R$ 1.159.990.000,00","R$ 6,16",8.50,1.67,6.70,9.6,"R$ 1.089.490.000,00","R$ 398.031.000,00"
5,WHRL4,"-R$ 1.159.990.000,00","R$ 6,52",8.99,1.67,7.14,9.6,"R$ 1.089.490.000,00","R$ 398.031.000,00"
6,GRND3,"-R$ 1.119.310.000,00","R$ 8,09",12.14,4.54,12.47,25.7,"R$ 601.005.000,00","R$ 230.609.000,00"


In [17]:
import win32com.client as win32
import pathlib

# Inserindo arquivo a ser anexado
anexo = pathlib.Path('../Analise/relatorio.csv')
anexo_ab = str(anexo.absolute())

# Criar a integração com o outlook
outlook = win32.Dispatch('outlook.application')

# Criar um email
email = outlook.CreateItem(0)

# Configurar as informações do seu e-mail
email.To = os.getenv("EMAIL")
email.Subject = " Relatório - Ações listadas na Bolsa "
email.HTMLBody = f"""

<p>Olá,</p>
<br>
<p>Segue em anexo o relatório solicitado.</p>
<br>
<p>Atenciosamente,</p>
<p>Lucas Alves.</p>
<p>Analista de Dados</p>
<br>
<img src="../Image/1-relatorio-gerencial.jpg">

"""

# Configurar anexo
email.Attachments.Add(anexo_ab)

# Envia o e-mail
email.Send()
print("Email Enviado")

Email Enviado
